# load packages

In [ ]:
library(tidyverse)

# read in input files

In [ ]:
vcf_id = read.csv('rosmap/ID_mapping/ROSMAP_VCF_IDs.txt',
                  header = F)
head(vcf_id)

In [ ]:
rnaseq_id = read.csv('rosmap/ID_mapping/ROSMAP_RNAseq_IDs.txt')
head(rnaseq_id)

In [ ]:
biospec = read.csv('rosmap/MetaData/ROSMAP_biospecimen_metadata.csv')
head(biospec)

In [ ]:
wgs_data = read.csv('rosmap/Metadata/ROSMAP_assay_wholeGenomeSeq_metadata.csv')
head(wgs_data)

In [ ]:
wgs_qc = read.csv('rosmap/Whole_Genome_Sequencing/WGS_sample_QC_info.csv')
head(wgs_qc)
nrow(wgs_qc)

In [ ]:
pheno = read.csv('rosmap/Metadata/ROSMAP_clinical.csv')
head(pheno)

In [ ]:
array = read.csv('rosmap/Genomic_Variants_SNP_Array/Raw/ROSMAP_arrayGenotype.fam',
                 sep = '',
                 header = F)
head(array)

In [ ]:
adsp = read.csv('ADSP/MAP_FILES/ROSMAP_ADSPID_mapping.csv')
head(adsp)
nrow(adsp)

In [ ]:
adsp_wgs_list = read.csv('rosmap/ID_mapping/ROSMAP_ADSP_WGS_samples.txt',
                         sep = '',
                         header = F)
head(adsp_wgs_list)

In [ ]:
adsp_pheno = read.csv('ADSP/v11_122023/v11_CombinedPhenotypes/ADSPIntegratedPhenotypes_DS_2023.08.08.csv')
head(adsp_pheno)

In [ ]:
adsp_qc = read.csv('/project/ritchie/datasets/ADSP/v11_122023/231211-ADSPv11_Release/downloads/v11-dl-dir/gcad.r4.wgs.36361.VCPA1.1.2022.08.15.v3.qcmetrics.seq.ALL.txt',
                   sep = '\t')
head(adsp_qc)

In [ ]:
methylation = read.csv('/project/ritchie/projects/AD_KMI/pathway_score/rosmap/ID_mapping/ROSMAP.Methylation.IDs.txt')
nrow(methylation)
length(unique(methylation$TargetID))
head(methylation)

# remove WGS samples that didn't pass QC

## filter to samples that failed

In [ ]:
unique(wgs_qc$QC)

In [ ]:
wgs_qc %>%
filter(grepl('Fail',QC)) %>%
filter(grepl('identical',QC))

In [ ]:
wgs_fail_qc = wgs_qc %>%
filter(grepl('Fail',QC))
head(wgs_fail_qc)
nrow(wgs_fail_qc)

## remove samples

In [ ]:
vcf_pass_qc=data.frame(vcf_id[!(vcf_id$V1 %in% wgs_fail_qc$WGS_id),])
colnames(vcf_pass_qc) = c('ID')
head(vcf_pass_qc)
nrow(vcf_pass_qc)
nrow(vcf_id)

# fix duplicates

## filter to duplicates

In [ ]:
wgs_dup = wgs_qc %>%
filter(grepl('identical', Duplicate)) %>%
filter(!grepl('Fail', QC))
nrow(wgs_dup)
print(wgs_dup)

## check to see if dups still exist even after removing samples that failed QC

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CTDVR',WGS_id))%>%
head()

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CTEGE',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CJEI2',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CJEL3',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CTEGU',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CTEDF',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CTEEX',WGS_id))

In [ ]:
wgs_qc%>%
filter(!grepl('Fail',QC))%>%
filter(grepl('SM-CJFKP',WGS_id))

## see how many samples have RNAseq based on the table

In [ ]:
wgs_qc %>%
filter(!grepl('Fail', QC)) %>%
filter(is.na(Concordant_with_RNAseq) == F) %>%
head()
wgs_qc%>%
filter(!grepl('Fail', QC)) %>%
filter(is.na(Concordant_with_RNAseq) == F) %>%
nrow()

# create vcf individual ID map

## filter to individuals in VCF pass QC file

In [ ]:
biospec_vcf = biospec[biospec$specimenID %in% vcf_pass_qc$ID,]
nrow(biospec_vcf)
length(unique(biospec_vcf$individualID))
nrow(vcf_pass_qc)
head(biospec_vcf)

## check out duplicates

In [ ]:
biospec_vcf_id = biospec_vcf %>%
select(individualID)
head(biospec_vcf_id)

In [ ]:
write.table(biospec_vcf_id,
            'rosmap/ID_mapping/biospec_vcf_pass_qc_poss_dups.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

In [ ]:
biospec_vcf_id_dup_list = read.csv('rosmap/ID_mapping/biospec_vcf_pass_qc_dups.txt',
                                   header = F)
head(biospec_vcf_id_dup_list)

In [ ]:
biospec_vcf_id_dups = biospec_vcf[biospec_vcf$individualID %in% biospec_vcf_id_dup_list$V1,]
nrow(biospec_vcf_id_dups)
biospec_vcf_id_dups %>%
arrange(individualID)

## fix duplicates

### filter to duplicates

In [ ]:
wgs_qc_dup = wgs_qc[wgs_qc$WGS_id %in% biospec_vcf_id_dups$specimenID,]
wgs_qc_dup

### keep duplicate with highest GQN

In [ ]:
wgs_qc_dup_fixed = wgs_qc_dup %>%
filter(grepl('SM-CJK4Y|SM-CTEE2|SM-CTEN3|SM-CTED9|SM-CTEIJ|SM-CTEMN', WGS_id))
wgs_qc_dup_fixed

### remove dups from biospecimen file

In [ ]:
biospec_vcf_no_dups = biospec_vcf[!(biospec_vcf$individualID %in% biospec_vcf_id_dup_list$V1),]
nrow(biospec_vcf_no_dups)
nrow(biospec_vcf)

### filter to fixed dups in biospecimen file

In [ ]:
biospec_vcf_new_dups = biospec_vcf[biospec_vcf$specimenID %in% wgs_qc_dup_fixed$WGS_id,]
nrow(wgs_qc_dup_fixed)
nrow(biospec_vcf_new_dups)

### concatenate files

In [ ]:
biospec_vcf_dups_fixed = rbind(biospec_vcf_no_dups,biospec_vcf_new_dups)
nrow(biospec_vcf_dups_fixed)
length(unique(biospec_vcf_dups_fixed$individualID))

# create map for RNAseq sample

## get unique RNA seq samples

In [ ]:
rnaseq_unique = rnaseq_id%>%
distinct()
nrow(rnaseq_unique)

## check out duplicates

In [ ]:
write.table(rnaseq_id,'rosmap/ID_mapping/rnaseq_ids.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

In [ ]:
rnaseq_dups = read.csv('rosmap/ID_mapping/rnaseq_ids_dups.txt',
                       header = F)
head(rnaseq_dups)

In [ ]:
rnaseq_id[rnaseq_id$individualID %in% rnaseq_dups$V1,]

In [ ]:
biospec_rnaseq_dup = biospec[biospec$individualID %in% rnaseq_dups$V1,]
nrow(biospec_rnaseq_dup)
biospec_rnaseq_dup %>%
filter(grepl('rnaSeq', assay))

In [ ]:
biospec_rnaseq = biospec[biospec$individualID %in% rnaseq_id$individualID,]
biospec_rnaseq %>%
filter(grepl('rnaSeq', assay)) %>%
group_by(tissue) %>%
summarize(n())

## filter

In [ ]:
biospec_rnaseq = biospec_vcf_no_dups[biospec_vcf_no_dups$individualID %in% rnaseq_unique$individualID,]
nrow(biospec_rnaseq)
nrow(biospec_vcf_no_dups)
nrow(rnaseq_unique)

## check out IDs that didn't map

In [ ]:
rnaseq_mismatch = data.frame(rnaseq_id[!(rnaseq_id$individualID %in% biospec_rnaseq$individualID),])
colnames(rnaseq_mismatch) = c('individualID')
nrow(rnaseq_mismatch)

In [ ]:
unique(biospec$assay)

In [ ]:
biospec_mismatch = biospec[biospec$individualID %in% rnaseq_mismatch$individualID,]
head(biospec_mismatch)

In [ ]:
biospec_mismatch_wgs = biospec_mismatch %>%
filter(grepl('wholeGenomeSeq', assay))
nrow(biospec_mismatch_wgs)

In [ ]:
biospec_mismatch_wgs_pass_qc=biospec_mismatch_wgs[!(biospec_mismatch_wgs$specimenID %in% wgs_fail_qc$WGS_id),]
nrow(biospec_mismatch_wgs_pass_qc)

In [ ]:
biospec_mismatch_wgs_no_dups=biospec_mismatch_wgs_pass_qc[!(biospec_mismatch_wgs_pass_qc$individualID %in% biospec_vcf_id_dup_list$V1),]
nrow(biospec_mismatch_wgs_no_dups)

In [ ]:
biospec_mismatch_wgs_dups = biospec_mismatch_wgs_pass_qc[biospec_mismatch_wgs_pass_qc$individualID %in% biospec_vcf_id_dup_list$V1,]
biospec_mismatch_wgs_dups %>%
arrange(individualID)

In [ ]:
biospec_mismatch_wgs_no_dups
nrow(biospec_mismatch_wgs_no_dups)

## see if we can fix IDs with pheno file

In [ ]:
nrow(pheno)

In [ ]:
pheno_mismatch = pheno[pheno$individualID %in% biospec_mismatch_wgs_no_dups$individualID,]
nrow(pheno_mismatch)
head(pheno_mismatch)

In [ ]:
pheno_mismatch_sub = pheno_mismatch %>%
select(projid,Study,individualID)
head(pheno_mismatch_sub)

In [ ]:
pheno_mismatch_wgs = merge(pheno_mismatch_sub,wgs_qc, by = 'projid')
nrow(pheno_mismatch_wgs)
head(pheno_mismatch_wgs)

In [ ]:
nrow(data.frame(vcf_id[vcf_id$V1 %in% pheno_mismatch_wgs$WGS_id,]))

In [ ]:
pheno_mismatch_wgs_sub = pheno_mismatch_wgs %>%
select(individualID,WGS_id)

In [ ]:
biospec_fixed = merge(biospec_mismatch_wgs_no_dups, pheno_mismatch_wgs_sub, by = 'individualID')
nrow(biospec_fixed)
head(biospec_fixed)

In [ ]:
biospec_fixed_sub = biospec_fixed %>%
select(individualID,
       WGS_id,
       specimenIdSource,
       samplingDate,
       organ,
       tissue,
       BrodmannArea,
       sampleStatus,
       tissueWeight,
       tissueVolume,
       nucleicAcidSource,
       cellType,
       fastingState,
       isPostMortem,
       samplingAge,
       visitNumber,
       assay,
       exclude,
       excludeReason) %>%
rename('specimenID' = 'WGS_id')
head(biospec_fixed_sub)

## concatenate to make final RNAseq map

In [ ]:
rnaseq_map_final = rbind(biospec_rnaseq, biospec_fixed_sub)
nrow(rnaseq_map_final)
head(rnaseq_map_final)

## subset to IDs

In [ ]:
rnaseq_map_final_sub = rnaseq_map_final %>%
select(individualID, specimenID)

## create ID list for plink score

In [ ]:
rnaseq_map_final_id = rnaseq_map_final_sub %>%
select(specimenID)

In [ ]:
nrow(rnaseq_map_final_sub[rnaseq_map_final_sub$specimenID %in% vcf_id$V1,])
nrow(rnaseq_map_final_sub)

## export

In [ ]:
write.table(rnaseq_map_final_sub,
            'rosmap/ID_mapping/ROSMAP_RNA_seq_WGS_ID_map.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

In [ ]:
write.table(rnaseq_map_final_id,
            'rosmap/ID_mapping/ROSMAP_RNA_seq_WGS_sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

# create methylation id map

## filter biospec file

In [ ]:
biospec_methyl = biospec[biospec$specimenID %in% methylation$TargetID,]
nrow(biospec_methyl)
length(unique(biospec_methyl$individualID))
length(unique(biospec_methyl$specimenID))
head(biospec_methyl)

## subset to IDs

In [ ]:
biospec_methyl_sub = biospec_methyl %>%
select(individualID, specimenID)
head(biospec_methyl_sub)

## export

In [ ]:
write.table(biospec_methyl_sub,
            'rosmap/ID_mapping/ROSMAP.Methylation_Array.individualID_map.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

# create VCF map for all samples

## subset map

In [ ]:
biospec_vcf_dups_fixed_sub = subset(biospec_vcf_dups_fixed, select = c(individualID, specimenID))
head(biospec_vcf_dups_fixed_sub)

## find samples that didn't match

In [ ]:
vcf_no_match = data.frame(vcf_pass_qc[!(vcf_pass_qc$ID %in% biospec_vcf_dups_fixed$specimenID),])
colnames(vcf_no_match) = 'ID'
nrow(vcf_no_match)
head(vcf_no_match)

## find these samples in phenotype file

In [ ]:
pheno_id = pheno %>%
select(projid, Study, individualID) %>%
mutate(specimenID = paste0(Study, projid))
head(pheno_id)

In [ ]:
pheno_vcf_no_match = pheno_id[pheno_id$specimenID %in% vcf_no_match$ID,]
nrow(pheno_vcf_no_match)

## subset pheno map

In [ ]:
pheno_vcf_no_match_sub = pheno_vcf_no_match %>%
select(individualID, specimenID)

## remove samples in VCF map (IDK HOW THEY GOT THERE OKAY)

In [ ]:
pheno_vcf_def_no_match_sub = pheno_vcf_no_match_sub[!(pheno_vcf_no_match_sub$individualID %in% biospec_vcf_dups_fixed_sub$individualID),]
nrow(pheno_vcf_def_no_match_sub)

## concatenate

In [ ]:
vcf_map_final = rbind(biospec_vcf_dups_fixed_sub, pheno_vcf_def_no_match_sub)
head(vcf_map_final)
nrow(vcf_map_final)
length(unique(vcf_map_final$individualID))

In [ ]:
nrow(biospec_vcf_dups_fixed_sub)
length(unique(biospec_vcf_dups_fixed_sub$individualID))

In [ ]:
nrow(pheno_vcf_no_match_sub)
length(unique(pheno_vcf_no_match_sub$individualID))

In [ ]:
test = pheno_vcf_no_match_sub[pheno_vcf_no_match_sub$individualID %in% biospec_vcf_dups_fixed_sub$individualID,]
nrow(test)

## create sample list

In [ ]:
vcf_map_final_sample = vcf_map_final %>%
select(specimenID)

## export

In [ ]:
write.table(vcf_map_final_sample,
            'rosmap/ID_mapping/ROSMAP.ALL_WGS.pass_qc.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

In [ ]:
write.table(vcf_map_final,
            'rosmap/ID_mapping/ROSMAP.ALL_WGS.pass_qc.ID_map.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

# create adsp map

## clean adsp file

In [ ]:
adsp_id = adsp %>%
mutate(specimenID = paste0(cohort_key, subject_key_original)) %>%
select(subject_key, specimenID)%>%
rename('ADSP_ID' = 'subject_key')
head(adsp_id)
nrow(adsp_id)

## map to pheno file

In [ ]:
head(pheno_id)

In [ ]:
adsp_rosmap = merge(pheno_id, adsp_id, by = 'specimenID')
nrow(adsp_rosmap)
nrow(adsp)
head(adsp_rosmap)

## filter mapped file

In [ ]:
adsp_rosmap_sub = adsp_rosmap %>%
select(individualID, ADSP_ID)
head(adsp_rosmap_sub)

## map to vcf file

In [ ]:
head(vcf_map_final)
nrow(vcf_map_final)

In [ ]:
adsp_rosmap_map = merge(vcf_map_final, adsp_rosmap_sub, by = 'individualID')
nrow(adsp_rosmap_map)
nrow(adsp_rosmap_sub)
nrow(vcf_map_final)
length(unique(adsp_rosmap_map$individualID))
head(adsp_rosmap_map)

## check if mapped IDs are all in ADSP WGS list

In [ ]:
adsp_pheno_id = adsp_pheno %>%
select(SampleID, SUBJID)
head(adsp_pheno_id)

In [ ]:
adsp_pheno_rosmap_wgs = adsp_pheno[adsp_pheno$SampleID %in% adsp_wgs_list$V1,]
nrow(adsp_pheno_rosmap_wgs)
nrow(adsp_wgs_list)

In [ ]:
adsp_rosmap_map_wgs = adsp_rosmap_map[adsp_rosmap_map$ADSP_ID %in% adsp_pheno_rosmap_wgs$SUBJID,]
nrow(adsp_rosmap_map_wgs)
head(adsp_rosmap_map_wgs)

In [ ]:
adsp_rosmap_map_wgs_rename = adsp_rosmap_map_wgs %>%
rename('SUBJID'='ADSP_ID')
adsp_rosmap_map_wgs_all_id = adsp_rosmap_map_wgs_rename %>%
left_join(adsp_pheno_id, by = 'SUBJID')%>%
select(individualID, specimenID, SampleID, SUBJID)
head(adsp_rosmap_map_wgs_all_id)
nrow(adsp_rosmap_map_wgs_all_id)

In [ ]:
write.table(adsp_rosmap_map_wgs_all_id,
            'rosmap/ID_mapping/ROSMAP.ADSP_WGS.ID_map.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

In [ ]:
adsp_rosmap_map_pheno = adsp_rosmap_map[adsp_rosmap_map$ADSP_ID %in% adsp_pheno$SUBJID,]
nrow(adsp_rosmap_map_pheno)
head(adsp_rosmap_map_pheno)

In [ ]:
adsp_rosmap_map_pheno_rename = adsp_rosmap_map_pheno %>%
rename('SUBJID' = 'ADSP_ID')
adsp_rosmap_map_pheno_all_id = adsp_rosmap_map_pheno_rename %>%
left_join(adsp_pheno_id, by='SUBJID')%>%
select(individualID, specimenID, SampleID, SUBJID)
head(adsp_rosmap_map_pheno_all_id)
nrow(adsp_rosmap_map_pheno_all_id)

In [ ]:
adsp_rosmap_map_pheno_all_id %>%
select(specimenID_RNAseq) %>%
na.omit() %>%
nrow()

In [ ]:
adsp_rosmap_map_pheno_all_id %>%
select(specimenID_MethylArray) %>%
na.omit() %>%
nrow()

In [ ]:
write.table(adsp_rosmap_map_pheno_all_id,
            'rosmap/ID_mapping/ROSMAP.ADSP_PHENO.ID_map.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

In [ ]:
adsp_pheno %>%
filter(grepl('ROS|MAP',Cohort)) %>%
nrow()

In [ ]:
adsp_pheno %>%
filter(!grepl('ROS|MAP',Cohort)) %>%
distinct(Cohort)

## check out ids that didn't map

### ADSP IDs that aren't in VCF files

In [ ]:
adsp_no_map = adsp_rosmap_sub[!(adsp_rosmap_sub$ADSP_ID %in% adsp_rosmap_map$ADSP_ID),]
nrow(adsp_no_map)
length(unique(adsp_no_map$individualID))
length(unique(adsp_no_map$ADSP_ID))
head(adsp_no_map)

In [ ]:
head(biospec_vcf_dups_fixed)

In [ ]:
biospec_vcf_adsp_no_map = biospec_vcf_dups_fixed[biospec_vcf_dups_fixed$individualID %in% adsp_no_map$individualID,]
nrow(biospec_vcf_adsp_no_map)

In [ ]:
nrow(biospec)

In [ ]:
biospec_vcf_broken=biospec[!(biospec$individualID %in% biospec_vcf_dups_fixed$individualID),]
nrow(biospec_vcf_broken)
length(unique(biospec_vcf_broken$individualID))

In [ ]:
adsp_broken_vcf=adsp_no_map[adsp_no_map$individualID %in% biospec_vcf_broken$individualID,]
nrow(adsp_no_map)

In [ ]:
biospec_vcf_broken_adsp=biospec_vcf_broken[biospec_vcf_broken$individualID %in% adsp_no_map$individualID,]
head(biospec_vcf_broken_adsp)

In [ ]:
unique(biospec_vcf_broken_adsp$assay)

In [ ]:
biospec_vcf_broken_adsp %>%
filter(grepl('wholeGenomeSeq', assay)) %>%
head()

### mismatch

#### create mismatch file

In [ ]:
adsp_mismatch = vcf_map_final[!(vcf_map_final$individualID %in% adsp_rosmap_sub$individualID),]
nrow(adsp_mismatch)
head(adsp_mismatch)

#### check for leading zeros

In [ ]:
adsp %>%
select(subject_key_original) %>%
filter(grepl('^0', subject_key_original)) %>%
nrow()

In [ ]:
pheno_id %>%
select(projid) %>%
filter(grepl('^0', projid)) %>%
nrow()

#### check if all IDs are SM

In [ ]:
adsp_mismatch %>%
filter(grepl('SM-', specimenID)) %>%
nrow()

In [ ]:
adsp_mismatch %>%
filter(!grepl('SM-', specimenID)) %>%
head()

#### see if individualID ares in pheno file

In [ ]:
pheno_mismatch = pheno_id[pheno_id$individualID %in% adsp_mismatch$individualID,]
nrow(pheno_mismatch)
head(pheno_mismatch)

In [ ]:
adsp_mismatch_rename = adsp_mismatch %>%
rename('vcf_specimenID' = 'specimenID')
pheno_adsp_mismatch = merge(adsp_mismatch_rename,pheno_mismatch, by = 'individualID')
nrow(pheno_adsp_mismatch)
head(pheno_adsp_mismatch)

In [ ]:
adsp_test=pheno_adsp_mismatch[pheno_adsp_mismatch$individualID %in% biospec_vcf_broken_adsp$individualID,]
nrow(adsp_test)
nrow(adsp_no_map)

In [ ]:
adsp_mismatch_fixed = merge(pheno_adsp_mismatch,adsp_rosmap, by = 'individualID')
nrow(adsp_mismatch_fixed)
head(adsp_mismatch_fixed)

In [ ]:
adsp_test = pheno_adsp_mismatch[pheno_adsp_mismatch$individualID %in% biospec$individualID,]
nrow(adsp_test)
head(adsp_test)

In [ ]:
head(adsp)
nrow(adsp)

# see if ROSMAP is in keep quest comb in ADSP

In [ ]:
adsp_qc_sub = adsp_qc %>%
select(Study, Sample.name) %>%
rename('SampleID' = 'Sample.name')
head(adsp_qc_sub)
print(nrow(adsp_qc_sub))

In [ ]:
adsp_pheno_study = adsp_pheno %>%
select(SampleID,Cohort)
head(adsp_pheno_study)
print(nrow(adsp_pheno_study))

In [ ]:
adsp_pheno_qc = left_join(adsp_pheno_study, adsp_qc_sub, by = 'SampleID')
head(adsp_pheno_qc)
nrow(adsp_pheno_qc)

In [ ]:
adsp_pheno_qc_rosmap = adsp_pheno_qc %>%
filter(grepl('ROS|MAP', Cohort))
head(adsp_pheno_qc_rosmap)
nrow(adsp_pheno_qc_rosmap)

In [ ]:
unique(adsp_pheno_qc_rosmap$Study)